In [ ]:

!pip install pyspark



In [ ]:
from pyspark.sql import SparkSession

# Spark oturumu başlatma
spark = SparkSession.builder.appName("PySparkGoogleColab").getOrCreate()

# Spark sürümünü kontrol etme
spark.version


'3.5.5'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#Google drive’a bağlanma


Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/sale.parquet'
#Dosya yolunu belirleme


In [ ]:
from google.colab import files
uploaded = files.upload()
#Dosya yükleme

Saving retailer.parquet to retailer.parquet
Saving customer.parquet to customer.parquet
Saving date.parquet to date.parquet
Saving product.parquet to product.parquet
Saving purchase.parquet to purchase.parquet
Saving region.parquet to region.parquet
Saving supplier.parquet to supplier.parquet


In [ ]:
from google.colab import files
uploaded = files.upload()
#Dosya yükleme


Saving region.parquet to region (1).parquet


In [ ]:
# Verileri yükleme
sale_df = spark.read.parquet('/content/drive/My Drive/sale.parquet')
region_df = spark.read.parquet('/content/drive/My Drive/region.parquet')

# Veri setlerinin ilk birkaç satırına bakma
sale_df.show()
region_df.show()



+--------+----------+-----------+-----------+--------+---------+----------+
|order_id|product_id|customer_id|retailer_id|quantity|total_amt|      date|
+--------+----------+-----------+-----------+--------+---------+----------+
|       1|       241|        551|          2|       1|       32|2023-08-30|
|       1|       139|        551|          2|       1|       23|2023-08-30|
|       1|        36|        551|          2|       1|       20|2023-08-30|
|       1|       319|        551|          2|       1|       41|2023-08-30|
|       1|         5|        551|          2|       1|       25|2023-08-30|
|       2|       163|        286|          1|       2|       80|2023-08-30|
|       2|       234|        286|          1|       2|       68|2023-08-30|
|       2|       311|        286|          1|       1|       35|2023-08-30|
|       3|       148|        889|          2|       3|      150|2023-06-30|
|       3|        88|        889|          2|       1|       10|2023-06-30|
|       4|  

In [ ]:
# Verileri yükleme
sale_df = spark.read.parquet('/content/drive/My Drive/sale.parquet')
region_df = spark.read.parquet('/content/drive/My Drive/region.parquet')

# Verilerin ilk 5 satırını gösterme
sale_df.show(5)
region_df.show(5)


+--------+----------+-----------+-----------+--------+---------+----------+
|order_id|product_id|customer_id|retailer_id|quantity|total_amt|      date|
+--------+----------+-----------+-----------+--------+---------+----------+
|       1|       241|        551|          2|       1|       32|2023-08-30|
|       1|       139|        551|          2|       1|       23|2023-08-30|
|       1|        36|        551|          2|       1|       20|2023-08-30|
|       1|       319|        551|          2|       1|       41|2023-08-30|
|       1|         5|        551|          2|       1|       25|2023-08-30|
+--------+----------+-----------+-----------+--------+---------+----------+
only showing top 5 rows

+-------+--------------+-----------------+
|city_id|     city_name|      region_name|
+-------+--------------+-----------------+
|      1|         Adana|       Ic Anadolu|
|      2|      Adiyaman|Guneydogu Anadolu|
|      3|Afyonkarahisar|       Ic Anadolu|
|      4|          Agri|     Dogu

In [ ]:
# Verileri region_id ile birleştirme
joined_df = sale_df.join(region_df, sale_df.retailer_id == region_df.city_id, 'inner')

# "Jacket" ürünlerini filtreleme (product_id'yi 241 olarak varsayalım)
jacket_sales_df = joined_df.filter(joined_df.product_id == 241)

# Bölgeye göre satışları gruplama ve toplam satış miktarlarını hesaplama
jacket_sales_per_region = jacket_sales_df.groupBy('region_name').agg({'quantity': 'sum', 'total_amt': 'sum'})

# Sonuçları gösterme
jacket_sales_per_region.show()


+-----------------+--------------+-------------+
|      region_name|sum(total_amt)|sum(quantity)|
+-----------------+--------------+-------------+
|Guneydogu Anadolu|          1344|           42|
|       Ic Anadolu|           832|           26|
|     Dogu Anadolu|            96|            3|
+-----------------+--------------+-------------+



In [ ]:
# DataFrame'i geçici bir görünüm olarak kaydetme
joined_df.createOrReplaceTempView("sales_and_regions")

# Spark SQL sorgusu ile ceket satışlarını bölgelere göre gruplama
sql_query = """
SELECT
    region_name,
    SUM(total_amt) AS total_sales,
    SUM(quantity) AS total_quantity
FROM sales_and_regions
WHERE product_id = 241
GROUP BY region_name
"""

# SQL sorgusunu çalıştırma ve sonucu gösterme
jacket_sales_sql = spark.sql(sql_query)
jacket_sales_sql.show()


+-----------------+-----------+--------------+
|      region_name|total_sales|total_quantity|
+-----------------+-----------+--------------+
|Guneydogu Anadolu|       1344|            42|
|       Ic Anadolu|        832|            26|
|     Dogu Anadolu|         96|             3|
+-----------------+-----------+--------------+



In [ ]:
# Bölgeye göre toplam ciroyu hesaplama
total_sales_per_region = sale_df.groupBy('retailer_id').agg({'total_amt': 'sum'}).withColumnRenamed('sum(total_amt)', 'total_sales')

# Her bölgenin adını almak için region_df ile birleştirme
total_sales_with_region = total_sales_per_region.join(region_df, total_sales_per_region.retailer_id == region_df.city_id, 'inner')

# En yüksek toplam ciroyu bulma
max_turnover_region = total_sales_with_region.orderBy('total_sales', ascending=False).limit(1)

# Sonucu gösterme
max_turnover_region.show()


+-----------+-----------+-------+---------+-----------------+
|retailer_id|total_sales|city_id|city_name|      region_name|
+-----------+-----------+-------+---------+-----------------+
|          2|     356278|      2| Adiyaman|Guneydogu Anadolu|
+-----------+-----------+-------+---------+-----------------+



In [ ]:
# Veri çerçevelerini geçici görünümler olarak kaydetme
sale_df.createOrReplaceTempView("sales")
region_df.createOrReplaceTempView("regions")

# SQL sorgusu ile bölgeye göre toplam ciroyu hesaplama
sql_query = """
    SELECT
        r.region_name,
        SUM(s.total_amt) AS total_sales
    FROM
        sales s
    JOIN
        regions r ON s.retailer_id = r.city_id
    GROUP BY
        r.region_name
    ORDER BY
        total_sales DESC
    LIMIT 1
"""

# SQL sorgusunu çalıştırma ve sonucu gösterme
max_turnover_region_sql = spark.sql(sql_query)
max_turnover_region_sql.show()


+-----------------+-----------+
|      region_name|total_sales|
+-----------------+-----------+
|Guneydogu Anadolu|     356278|
+-----------------+-----------+

